In [190]:
#!/usr/bin/env python3.10
# coding: utf-8

import logging
import pandas as pd

#setup logger
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%I:%M:%S')

#import dpd csv as dataframe
logging.warning(f"opening dpd.csv as data frame")
csv_file = "/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs/dpd.csv"
df = pd.read_csv (csv_file, sep="\t", dtype={"Metadata":str, "POS = Grammar":str, "Grp":str})
df["Grp"].fillna("", inplace=True)
df["Grp"].str.replace(fr"(\d)\.0", "\\1")
# df.fillna("", inplace=True)

08:58:42 opening dpd.csv as data frame
/tmp/ipykernel_10580/603085108.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Grp"].str.replace(fr"(\d)\.0", "\\1")


0         
1         
2         
3         
4         
        ..
44740     
44741     
44742     
44743     
44744     
Name: Grp, Length: 44745, dtype: object

In [191]:
# #combine meaning and buddhadatta columns
# df = pd.read_csv (csv_file, sep="\t", dtype={"Metadata":str, "POS = Grammar":str})

# df["Buddhadatta"] = df["Buddhadatta"].str.replace("(.+)", "<div style='color: #AA4400'>\\1<div>")



# df.loc[df["Meaning IN CONTEXT"].isnull(), "Meaning IN CONTEXT"] = df["Buddhadatta"]


In [192]:
# extract root families
logging.warning("extracting root family names")

root_family_list = df[~df["Family"].isnull()]
root_family_list = root_family_list[~root_family_list["Pāli1"].str.contains("Help", "Abbreviations")]
root_family_list = root_family_list[["Pāli Root", "Grp", "Root Meaning", "Family"]]
root_family_list = root_family_list.drop_duplicates(subset=["Pāli Root", "Grp", "Root Meaning", "Family"])
root_family_list.sort_values(["Pāli Root", "Grp", "Root Meaning", "Family"], ascending = (True, True, True, True), inplace=True)
root_family_list = root_family_list.reset_index(drop=True)



# write families
logging.warning(f"/csvs/writing root families list.csv")
root_family_list.to_csv("/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/root families list.csv", sep="\t")

# row count
root_families_count = root_family_list.shape[0]
logging.warning(f"root_families_count = {root_families_count}")


08:58:42 extracting root family names
08:58:42 /csvs/writing root families list.csv
08:58:42 root_families_count = 2630


In [193]:
yn = input("for (s)tudents or (r)oots diciotnary or (a)nki ")

In [194]:
# for study

if yn == "s":

    #writing root families study.csv
    logging.warning ("writing root families for study.csv")
    txt_file = open ("/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/csvs/root families for study.csv", 'w', encoding= "'utf-8")
    txt_file.write(f"Family\tGrp\tRoot Meaning\tPāli\tConstruction\tPOS\tMeaning\n")

    logging.warning ("~" *40)
    row =0

    for row in range (0, root_families_count):
        root = root_family_list.iloc[row, 0]
        root_group = (root_family_list.iloc[row, 1]) #.astype(int)
        root_meaning = root_family_list.iloc[row, 2]
        root_family = root_family_list.iloc[row, 3]

        if row % 100 == 0:
            logging.warning (f"{row} {root} {root_group} {root_meaning} {root_family}")

        test1 = ~df["Pāli Root"].isnull()
        test2 = df["Pāli Root"] == (root)
        test3 = df["Grp"] == (root_group).astype(int)
        test4 = df["Root Meaning"] == (root_meaning)
        test5 = df["Family"] == (root_family)

        filter = test1 & test2 & test3 & test4 & test5
        filtered_df = df.loc[filter, ["Family", "Grp", "Root Meaning", "Pāli1", "Construction", "POS", "Meaning IN CONTEXT"]]

        with open("/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/csvs/root families for study.csv", 'a') as txt_file:
            txt_file.write (f"~~~~~~~~~~\n")
            # txt_file.write (f"{root_family}\n")
            filtered_df.to_csv(txt_file, header=False, index=False, sep="\t")



In [195]:
#  for anki

from datetime import date

today = date.today()
date = today.strftime("%d")

if yn == "a":

    #combine meaning and buddhadatta columns
    
    df["Buddhadatta"] = df["Buddhadatta"].str.replace("(.+)", "<div style='color: #AA4400'>\\1<div>")

    df.loc[df["Meaning IN CONTEXT"].isnull(), "Meaning IN CONTEXT"] = df["Buddhadatta"]

    df["Construction"] = df["Construction"].str.replace("(.+)\n.+", "\\1")

    #writing root families for anki.csv
    logging.warning ("writing root families for anki.csv")
    txt_file = open ("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs for anki/root families.csv", 'w', encoding= "'utf-8")

    

    logging.warning ("~" *40)
    row =0

    for row in range (0, root_families_count):
        root = root_family_list.iloc[row, 0]
        root_group = (root_family_list.iloc[row, 1]) #.astype(int)
        root_meaning = root_family_list.iloc[row, 2]
        root_family = root_family_list.iloc[row, 3]

        if row % 500 == 0:
            logging.warning (f"{row} {root} {root_group} {root_meaning} {root_family}")
        
        test1 = ~df["Pāli Root"].isnull()
        test2 = df["Pāli Root"] == (root)
        test3 = df["Grp"] == (root_group) #.astype(int)
        test4 = df["Root Meaning"] == (root_meaning)
        test5 = df["Family"] == (root_family)

        filter = test1 & test2 & test3 & test4 & test5
        filtered_df = df.loc[filter, ["Pāli1", "POS", "Meaning IN CONTEXT", "Construction"]]

        # filtered_df['Pāli1'] = "<tr valign='top'><div style='color: #FFB380'><td>" + df['Pāli1']

        with open("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs for anki/root families.csv", 'a') as txt_file:
            txt_file.write(f"<b>{root_family}</b> {root_group} ({root_meaning})\t")
            txt_file.write(f"<table><tbody>")
            for row in range(filtered_df.shape[0]):
                pāli = filtered_df.iloc[row, 0]
                pos = filtered_df.iloc[row, 1]
                meaning = filtered_df.iloc[row, 2]
                construction = filtered_df.iloc[row, 3]
                txt_file.write(f"<tr valign='top'><div style='color: #FFB380'><td>{pāli}</td><td><div style='color: #FF6600'>{pos}</div></td><td><div style='color: #FFB380'>{meaning}</td><td><div style='color: #AA4400'>{construction}</div></td></tr>")
            txt_file.write(f"</tbody></table>")
            txt_file.write(f"\t{date}")
            txt_file.write(f"\n")



/tmp/ipykernel_10580/2758916074.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Buddhadatta"] = df["Buddhadatta"].str.replace("(.+)", "<div style='color: #AA4400'>\\1<div>")
/tmp/ipykernel_10580/2758916074.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Construction"] = df["Construction"].str.replace("(.+)\n.+", "\\1")
08:58:44 writing root families for anki.csv
08:58:44 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
08:58:44 0 √acc 1 shine, praise √acc
08:58:49 500 √dā 3 tie, bind saṃ √dā
08:58:54 1000 √kas 1 drag, draw pari √kas
08:58:59 1500 √nam 1 bend ud √nam
08:59:04 2000 √sid 3 sweat, steam pari √sid
08:59:09 2500 √yas 3 strive, struggle adhi pa √yas


In [196]:
# for dictionary

if yn == "r":

    #writing root families for dictionary.csv
    logging.warning ("writing root families for dictionary.csv")
    txt_file = open ("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs/root families for dictionary.csv", 'w', encoding= "'utf-8")
    txt_file.write(f"Root\tGrp\tMeaning\tFamily\tPāli\tPOS\tEnglish\n")

    logging.warning ("~" *40)
    row =0

    for row in range (0, root_families_count):
        root = root_family_list.iloc[row, 0]
        root_group = (root_family_list.iloc[row, 1]).astype(int)
        root_meaning = root_family_list.iloc[row, 2]
        root_family = root_family_list.iloc[row, 3]

        if row % 100 == 0:
            logging.warning (f"{row} {root} {root_group} {root_meaning} {root_family}")

        test1 = ~df["Pāli Root"].isnull()
        test2 = df["Pāli Root"] == (root)
        test3 = df["Grp"] == (root_group).astype(int)
        test4 = df["Root Meaning"] == (root_meaning)
        test5 = df["Family"] == (root_family)

        filter = test1 & test2 & test3 & test4 & test5
        filtered_df = df.loc[filter, ["Pāli Root", "Grp", "Root Meaning", "Family", "Pāli1", "POS", "Meaning IN CONTEXT"]]

        with open("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs/root families for dictionary.csv", 'a') as txt_file:
            filtered_df.to_csv(txt_file, header=False, index=False, sep="\t")
